<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/06_combined_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-25 15:32:41--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12099 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.82K  --.-KB/s    in 0s      

2022-06-25 15:32:41 (81.3 MB/s) - ‘utils.py’ saved [12099/12099]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [1]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

In [2]:
BATCH_SIZE = 16
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [3]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [4]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [5]:
classes, class_weight = get_class_weight()

In [6]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [7]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
rgb_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
hsv_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)
xyz_hub_module = hub.KerasLayer(HUB_URL,  trainable=True)

rgb_features = rgb_hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = xyz_hub_module(xyz_input)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hsv_hub_module(hsv_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features, hsv_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [8]:
model._name = 'combined-model-v2'

In [9]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 500, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [10]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='val_auc', 
                                              verbose=1,
                                              patience=5,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+".tf")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='val_auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [11]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 1/15
1037/1037 [==============================] - ETA: 0s - loss: 18.4623 - auc: 0.7041 - accuracy: 0.6335

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2.tf/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2.tf/assets


1037/1037 [==============================] - 1916s 2s/step - loss: 18.4623 - auc: 0.7041 - accuracy: 0.6335 - val_loss: 1.1498 - val_auc: 0.7319 - val_accuracy: 0.6589
Epoch 2/15
1037/1037 [==============================] - ETA: 0s - loss: 11.4086 - auc: 0.8491 - accuracy: 0.7648

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2.tf/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2.tf/assets


1037/1037 [==============================] - 744s 716ms/step - loss: 11.4086 - auc: 0.8491 - accuracy: 0.7648 - val_loss: 0.8773 - val_auc: 0.7983 - val_accuracy: 0.7182
Epoch 3/15
1037/1037 [==============================] - ETA: 0s - loss: 8.3749 - auc: 0.9066 - accuracy: 0.8247

INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2.tf/assets


INFO:tensorflow:Assets written to: ./logs/checkpoints/combined-model-v2.tf/assets


1037/1037 [==============================] - 742s 715ms/step - loss: 8.3749 - auc: 0.9066 - accuracy: 0.8247 - val_loss: 0.6818 - val_auc: 0.8777 - val_accuracy: 0.7983
Epoch 4/15
1037/1037 [==============================] - 689s 664ms/step - loss: 6.2747 - auc: 0.9348 - accuracy: 0.8610 - val_loss: 0.8959 - val_auc: 0.8194 - val_accuracy: 0.7445
Epoch 5/15
1037/1037 [==============================] - 690s 665ms/step - loss: 4.3683 - auc: 0.9605 - accuracy: 0.9015 - val_loss: 0.7908 - val_auc: 0.8627 - val_accuracy: 0.8020
Epoch 6/15
1037/1037 [==============================] - 690s 665ms/step - loss: 2.9657 - auc: 0.9777 - accuracy: 0.9316 - val_loss: 0.8425 - val_auc: 0.8511 - val_accuracy: 0.8023
Epoch 7/15
1037/1037 [==============================] - 690s 664ms/step - loss: 1.8468 - auc: 0.9876 - accuracy: 0.9513 - val_loss: 0.8263 - val_auc: 0.8710 - val_accuracy: 0.8372
Epoch 8/15
1037/1037 [==============================] - 690s 665ms/step - loss: 1.1779 - auc: 0.9934 - accuracy

In [13]:
!cp  -r /content/logs/checkpoints/combined-model-v2.tf /content/drive/MyDrive/checkpoints/combined-v2-final

In [15]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=15,
                    initial_epoch=8,
                    validation_data=val_ds,
                    class_weight=class_weight)

Epoch 9/15
1037/1037 [==============================] - 691s 666ms/step - loss: 0.5545 - auc: 0.9971 - accuracy: 0.9850 - val_loss: 1.2829 - val_auc: 0.8208 - val_accuracy: 0.8037
Epoch 10/15
1037/1037 [==============================] - 692s 666ms/step - loss: 0.4024 - auc: 0.9978 - accuracy: 0.9905 - val_loss: 1.1353 - val_auc: 0.8413 - val_accuracy: 0.8206
Epoch 11/15
1037/1037 [==============================] - 693s 667ms/step - loss: 0.3355 - auc: 0.9986 - accuracy: 0.9914 - val_loss: 1.0920 - val_auc: 0.8485 - val_accuracy: 0.8295
Epoch 12/15
1037/1037 [==============================] - 693s 667ms/step - loss: 0.2336 - auc: 0.9989 - accuracy: 0.9941 - val_loss: 1.0458 - val_auc: 0.8572 - val_accuracy: 0.8392


KeyboardInterrupt: ignored